In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

/home/prem/NLP Practice/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 25.7M/25.7M [00:05<00:00, 5.28MB/s]

Extracting files...


Path to dataset files: /home/prem/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
# lowercase
df['review'] = df['review'].str.lower()

In [15]:
# to remove html tags
import re
def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [16]:
df['review'] = df['review'].apply(remove_html)

In [18]:
df['review'][1]

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

In [ ]:
# to remove urls
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [21]:
df['review'] = df['review'].apply(remove_url)

In [ ]:
# to remove punctuation (but this is slow method)
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
exclude = string.punctuation

In [23]:
def remove_punc(text):
    for i in exclude:
        text = text.replace(i, '')
    return text

In [24]:
df['review'] = df['review'].apply(remove_punc)

In [25]:
df['review'][1]

'a wonderful little production the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done'

In [26]:
# to remove punctuation (but this is fast method)
def remove_punc_fast(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [27]:
df['review'] = df['review'].apply(remove_punc_fast)

In [38]:
# to convert chat words to normal words

chat_words = {
    # Basic reactions
    "lol": "laughing out loud",           # [web:41][web:43]
    "lmao": "laughing my ass off",        # [web:45]
    "rofl": "rolling on the floor laughing",  # [web:41][web:43]
    "omg": "oh my god",                   # [web:43]
    "wtf": "what the fuck",               # [web:42]
    "wth": "what the hell",
    "brb": "be right back",               # [web:41]
    "gtg": "got to go",                   # [web:41][web:43]
    "afk": "away from keyboard",          # [web:41]
    "idk": "I don't know",                # [web:41][web:43]
    "idc": "I don't care",
    "ikr": "I know, right",               # [web:41]
    "tbh": "to be honest",                # [web:41][web:43]
    "imo": "in my opinion",               # [web:43]
    "imho": "in my humble opinion",       # [web:43]
    "btw": "by the way",                  # [web:43]
    "np": "no problem",                   # [web:43]
    "thx": "thanks",                      # [web:41]
    "ty": "thank you",                    # [web:41]
    "yw": "you're welcome",               # [web:41]
    "gg": "good game",
    "hf": "have fun",
    "gl": "good luck",

    # Short phrases / tone
    "fr": "for real",                     # [web:41]
    "rn": "right now",                    # [web:43]
    "smh": "shaking my head",             # [web:43]
    "ngl": "not gonna lie",               # [web:43]
    "no cap": "I’m serious / not lying",
    "cap": "lie / that’s not true",
    "lowkey": "a bit / secretly",
    "highkey": "very / openly",
    "af": "very / as fuck",               # [web:41]
    "tho": "though",
    "bc": "because",
    "cuz": "because",
    "jk": "just kidding",                 # [web:43]
    "bruh": "expression of disbelief / bro",
    "sus": "suspicious / sketchy",        # [web:43]

    # Social / messaging
    "dm": "direct message",               # [web:41][web:43]
    "pm": "private message",              # [web:41]
    "irl": "in real life",                # [web:43]
    "tldr": "too long; didn’t read",      # [web:46]
    "fyi": "for your information",        # [web:43]
    "omw": "on my way",                   # [web:43]
    "afaik": "as far as I know",          # [web:43]
    "bbl": "be back later",
    "bday": "birthday",
    "nvm": "never mind",                  # [web:45][web:43]
    "ttyl": "talk to you later",          # [web:41][web:43]
    "asap": "as soon as possible",        # [web:41]
    "hbd": "happy birthday",              # [web:41]
    "sup": "what's up?",                  # [web:41]

    # Feelings / emphasis
    "mb": "my bad",                       # [web:41]
    "sry": "sorry",                       # [web:41]
    "imu": "I miss you",                  # [web:41]
    "fomo": "fear of missing out",        # [web:43]
    "tmi": "too much information",        # [web:46]
    "tbf": "to be fair",                  # [web:43]
    "ofc": "of course",                   # [web:43]
}

In [39]:
def chat_word_to_normal(text):
    for word in chat_words:
        text = text.replace(word, chat_words[word])
    return text

In [40]:
chat_word_to_normal("idk what to do tbh, smh")

"I don't know what to do to be honest, shaking my head"

In [41]:
df['review'] = df['review'].apply(chat_word_to_normal)

In [43]:
# spelling correction

from textblob import TextBlob

In [47]:
# to remove stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/prem/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [48]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word not in stopwords.words('english'):
            new_text.append(word)
    return ' '.join(new_text)

In [49]:
df['review'] = df['review'].apply(remove_stopwords)

In [50]:
df['review'][1]

'wonderful little production filming technique unassuming oldtimebbecause fashion gives comforting sometimes discomforting sense realism entire piece actors extremely well chosen michael sheen got polari voices pat truly see seamless editing guided references williams diary entries well worth watching terrificly written performed piece masterful production one great masters comedy life realism really comes home little things fantasy guard rather use traditional dream techniques remains solid disappears plays knowledge senses particularly scenes conceright nowing orton halliwell sets particularly flat halliwells murals decorating every surface terribly well done'

In [51]:
# handling emoji (remove or convert to text)

# removing emoji
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [53]:
# replace emoji with text
import emoji
def emoji_to_text(text):
    return emoji.demojize(text)

In [55]:
df['review'] = df['review'].apply(emoji_to_text)

In [ ]:
# Tokenization

# split function (not recommended for real use)
def tokenize(text):
    return text.split()

In [56]:
# tokenization using re (but this is not recommended for real use)
def tokenize_re(text):
    pattern = re.compile(r'\w+')
    return pattern.findall(text)

In [58]:
# tokenization using nltk (recommended for real use)
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")

df['review'] = df['review'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /home/prem/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/prem/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [59]:
df['review']

0        [one, reviewers, mentioned, /, fuckter, watchi...
1        [wonderful, little, production, filming, techn...
2        [thoughught, wonderful, way, spend, time, hot,...
3        [basically, theres, family, little, boy, jake,...
4        [petter, matteis, love, time, money, visually,...
                               ...                        
49995    [thoughught, movie, right, good, job, wasnt, c...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [cathoughlic, taught, parochial, elementary, s...
49998    [im, going, disagree, previous, comment, side,...
49999    [one, expects, star, trek, movies, high, art, ...
Name: review, Length: 50000, dtype: object

In [60]:
# you can also use spacy for tokenization, but it is slow and not recommended for real use

In [61]:
# stemming

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stem_text(text):
    return [stemmer.stem(word) for word in text]

In [62]:
df['review'] = df['review'].apply(stem_text)

In [ ]:
# lemmatization

In [65]:
# Download wordnet for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/prem/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/prem/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to apply lemmatization to text
def apply_lemmatization(text):
    """Apply lemmatization to tokenized text"""
    if isinstance(text, list):
        # If text is already tokenized, lemmatize each token
        return [lemmatizer.lemmatize(word) for word in text]
    else:
        # If text is a string, tokenize first then lemmatize
        tokens = word_tokenize(text.lower())
        return [lemmatizer.lemmatize(word) for word in tokens]

# Apply lemmatization to the 'review' column
df['review'] = df['review'].apply(apply_lemmatization)
